In [284]:
import os
import pandas as pd
import numpy as np
import json
import random

In [291]:
variable_descriptions = dict()
with open('data-dictionary.json') as f:
    data_dict = json.load(f)
    for var in data_dict['categories']['housing'] + data_dict['categories']['person']:
        variable_descriptions[var['name']] = var['short_description']

In [292]:
data_dir = 'data/csv'

In [738]:
housing_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if (f.endswith('.csv') and 'ss14h' in f)]
person_files = [f.replace('ss14h', 'ss14p') for f in housing_files]

sample = None

for h_fn, p_fn in zip(housing_files, person_files):
    print('\r' + p_fn, end='')
    with open(p_fn) as f:
        person_data = pd.read_csv(f, na_values=['','.',' '])
        person_data = person_data[person_data.columns[:129]]
        person_data.drop('SOCP', 1, inplace=True)
        person_data.drop('NAICSP', 1, inplace=True)
    print('\r' + h_fn, end='')
    with open(h_fn) as f:
        housing_data = pd.read_csv(f, na_values=['','.',' '])
        housing_data = housing_data[housing_data.columns[:102]]
    merged_data = pd.merge(housing_data, person_data, how='outer', on=['SERIALNO', 'PUMA', 'ST', 'ADJINC'], suffixes=['_H', '_P'])
    merged_data.WGTP = merged_data.WGTP / ((merged_data.NP == 0) + merged_data.NP)
    with open(p_fn) as f:
        sample = pd.concat([
            sample,
            merged_data.select(lambda x: random.random() < 0.02)
        ])
print('\rDone')

data/csv/ss14pfl.csv

CParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [570]:
def construct_bins(data):
    bins = data.quantile([.1,.2,.3,.4,.5,.6,.7,.8,.9])
    bins = pd.concat([
            pd.DataFrame(np.full((1, bins.shape[1]), -np.inf), columns=bins.columns),
            bins,
            pd.DataFrame(np.full((1, bins.shape[1]), np.inf), columns=bins.columns)
    ])
    return bins

In [571]:
def bin_data(data, bins):
    out_columns = []
    for i in range(len(data.columns)):
        column = data[data.columns[i]]
        column_bins = bins[bins.columns[i]]
        column_bins = column_bins.unique()

        binned_column = pd.cut(column, column_bins, labels=False)
        out_columns.append(binned_column)

    binned_data = pd.concat(out_columns, axis=1, keys=housing_sample.columns)
    binned_data = (binned_data + 1).fillna(0).astype('int')
    return binned_data

In [572]:
def construct_joint_matrix(data, bins):
    binned = bin_data(data, bins)
    m = len(binned.columns)
    out = np.zeros((m, m, 11 * 11))
    for i in range(m):
        for j in range(m):
            bins = binned[binned.columns[i]] + binned[binned.columns[j]] * 11
            counts = pd.value_counts(bins)
            out[i, j, counts.index] = counts.as_matrix()
    return out.reshape((m, m, 11, 11))

In [573]:
import scipy.stats
def mi(joint_matrix):
    m = len(joint_matrix)
    out = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            sub_matrix = joint_matrix[i, j, :, :]
            tot = sub_matrix.sum()
            if tot == 0:
                tot = 1.
            sub_matrix = sub_matrix / tot
            prob_i = sub_matrix.sum(0)
            prob_j = sub_matrix.sum(1)
            outer = np.dot(np.transpose([prob_i]), [prob_j]).T
            mi_i_j = scipy.stats.entropy(sub_matrix.ravel(), outer.ravel())
            out[i, j] = mi_i_j
            out[j, i] = mi_i_j
    return out

In [574]:
def entropy(joint_matrix):
    single = joint_matrix[:,0,:].sum(1)
    single[0, :] = joint_matrix[0, 1].sum(1)
    out = []
    for row in single:
        out.append(scipy.stats.entropy(row))
    return np.array(out)

# Housing

In [ ]:
housing_bins = construct_bins(housing_sample)

In [610]:
housing_joint = construct_joint_matrix(housing_sample, housing_bins)

In [611]:
housing_entropy = entropy(housing_joint)

high_entropy_columns = [
    c for c, H in reversed(list(zip(
                housing_data.columns[np.argsort(housing_entropy)],
                np.sort(housing_entropy)
        ))) if (H > 0.9 and c != 'WGTP')]

housing_quality_sample = housing_sample[high_entropy_columns]
housing_quality_bins = housing_bins[high_entropy_columns]

In [612]:
housing_joint = construct_joint_matrix(housing_quality_sample, housing_quality_bins)
housing_entropy = entropy(housing_joint)

In [613]:
housing_mi = mi(housing_joint)

In [614]:
housing_mi_weighted = housing_mi * np.outer(housing_entropy, housing_entropy)

In [615]:
elec_index = np.where(housing_quality_sample.columns == 'ELEP')[0][0]
elec_importance = housing_mi[elec_index, :]
pd.DataFrame([
        (c, variable_descriptions[c], imp)
        for imp, c in reversed(list(sorted(zip(
                        elec_importance,
                        housing_quality_sample.columns
                    ))))][:10], columns=['name', 'desc', 'electricity mi'])

,name,desc,electricity mi
0,ELEP,Electricity (monthly cost),2.067039
1,HHT,Household/family type,0.518024
2,VEH,Vehicles (1 ton or less) available,0.516167
3,WATP,Water (yearly cost),0.515054
4,TEN,Tenure,0.508448
5,HINCP,Household income (past 12 months),0.499185
6,GASP,Gas (monthly cost),0.491757
7,HUPAC,HH presence and age of children,0.489857
8,R18,Presence of persons under 18 years in househol...,0.489714
9,HUPARC,HH presence and age of related children,0.489604


In [616]:
hinc_importance = housing_mi[0, :]
pd.DataFrame([
        (c, variable_descriptions[c], imp)
        for imp, c in reversed(list(sorted(zip(
                        hinc_importance,
                        housing_quality_sample.columns
                    ))))][:10], columns=['name', 'desc', 'income mi'])

,name,desc,income mi
0,HINCP,Household income (past 12 months),2.086293
1,FINCP,Family income (past 12 months),1.064146
2,VEH,Vehicles (1 ton or less) available,0.570920
3,HHT,Household/family type,0.559342
4,ACCESS,Access to the Internet,0.540857
5,TEN,Tenure,0.540217
6,HANDHELD,Handheld computer,0.536748
7,R60,Presence of persons 60 years and over in house...,0.500602
8,ELEP,Electricity (monthly cost),0.499185
9,R65,Presence of persons 65 years and over in house...,0.496595


In [617]:
pd.DataFrame([
        (c, variable_descriptions[c], imp)
        for imp, c in reversed(list(sorted(zip(
                        housing_entropy,
                        housing_quality_sample.columns
                    ))))][:10], columns=['name', 'desc', 'entropy'])

,name,desc,entropy
0,WATP,Water (yearly cost),2.077109
1,HINCP,Household income (past 12 months),2.077109
2,ELEP,Electricity (monthly cost),2.067039
3,RMSP,Number of Rooms,1.914831
4,YBL,When structure first built,1.877680
5,VALP,Property value,1.817914
6,SMOCP,Selected monthly owner costs,1.808459
7,TAXP,Property taxes (yearly amount),1.800971
8,OCPIP,Selected monthly owner costs as a percentage o...,1.799918
9,GASP,Gas (monthly cost),1.797283


In [618]:
importance = housing_mi_weighted.sum(1) - np.diag(housing_mi_weighted)
pd.DataFrame([
        (c, variable_descriptions[c], imp)
        for imp, c in reversed(list(sorted(zip(
                        importance,
                        housing_quality_sample.columns
                    ))))][:15], columns=['name', 'desc', 'importance'])

,name,desc,importance
0,HINCP,Household income (past 12 months),47.858122
1,WATP,Water (yearly cost),42.692083
2,TEN,Tenure,41.900753
3,HHT,Household/family type,41.379235
4,ELEP,Electricity (monthly cost),40.619745
5,MV,When moved into this house or apartment,37.179012
6,WKEXREL,Work experience of householder and spouse,35.827582
7,SMOCP,Selected monthly owner costs,35.469233
8,FINCP,Family income (past 12 months),34.495940
9,GASP,Gas (monthly cost),34.271027


In [637]:
eigval, eigvec = np.linalg.eig(housing_mi)

In [641]:
x = range(len(housing_mi))

In [642]:
importance = np.sum(np.abs(np.reshape(eigval[:20], (20, 1)) * eigvec[:20]), 0)

In [643]:
pd.DataFrame([
        (c, variable_descriptions[c], imp)
        for imp, c in reversed(list(sorted(zip(
                        importance,
                        housing_quality_sample.columns
                    ))))][:15], columns=['name', 'desc', 'importance'])

,name,desc,importance
0,SMOCP,Selected monthly owner costs,14.133961
1,VALP,Property value,11.488619
2,INSP,Fire/hazard/flood insurance (yearly amount),9.704058
3,GASP,Gas (monthly cost),9.202845
4,HINCP,Household income (past 12 months),8.505409
5,OCPIP,Selected monthly owner costs as a percentage o...,8.330829
6,RMSP,Number of Rooms,7.741651
7,HUPAOC,HH presence and age of own children,7.597963
8,YBL,When structure first built,7.213832
9,ELEP,Electricity (monthly cost),6.575923


# Person

In [686]:
person_bins = construct_bins(person_sample)

In [687]:
person_joint = construct_joint_matrix(person_sample, person_bins)

In [703]:
person_sample.columns

Index(['ADJINC', 'PWGTP', 'AGEP', 'CIT', 'CITWP', 'COW', 'DDRS', 'DEAR',
       'DEYE', 'DOUT',
       ...
       'RACPI', 'RACSOR', 'RACWHT', 'RC', 'SCIENGP', 'SCIENGRLP', 'SFN', 'SFR',
       'VPS', 'WAOB'],
      dtype='object', length=122)

In [688]:
person_entropy = entropy(person_joint)

high_entropy_columns = [
    c for c, H in reversed(list(zip(
                person_sample.columns[np.argsort(person_entropy)],
                np.sort(person_entropy)
        ))) if (H > 0.5 and c != 'PWGTP')]

person_quality_sample = person_sample[high_entropy_columns]
person_quality_bins = person_bins[high_entropy_columns]

In [693]:
person_joint = construct_joint_matrix(person_quality_sample, person_quality_bins)
person_entropy = entropy(person_joint)

In [694]:
person_mi = mi(person_joint)

In [695]:
person_mi_weighted = person_mi * np.outer(person_entropy, person_entropy)

In [706]:
variable_descriptions['OCCP'] = variable_descriptions['OCCP4']
pd.DataFrame([
        (c, variable_descriptions[c], imp)
        for imp, c in reversed(list(sorted(zip(
                        person_entropy,
                        person_quality_sample.columns
                    ))))][:20], columns=['name', 'desc', 'entropy'])

,name,desc,entropy
0,AGEP,Age,2.301385
1,ADJINC,Adjustment factor for income and earnings doll...,2.301385
2,ANC1P,Recoded Detailed Ancestry - first entry,2.117054
3,OCCP,Occupation recode for 2012 and later based on ...,2.016660
4,MARHYP,Year last married,2.016027
5,INDP,Industry recode for 2013 and later based on 20...,2.010991
6,PERNP,Total person's earnings,1.967834
7,SCHL,Educational attainment,1.880831
8,WAGP,Wages or salary income past 12 months,1.872298
9,JWAP,Time of arrival at work - hour and minute,1.640358


In [708]:
importance = person_mi_weighted.sum(1) - np.diag(person_mi_weighted)

pd.DataFrame([
        (c, variable_descriptions[c], imp)
        for imp, c in reversed(list(sorted(zip(
                        importance,
                        person_quality_sample.columns
                    ))))][:20], columns=['name', 'desc', 'importance'])

,name,desc,importance
0,PERNP,Total person's earnings,51.107030
1,AGEP,Age,47.700910
2,WAGP,Wages or salary income past 12 months,45.838612
3,OCCP,Occupation recode for 2012 and later based on ...,31.847647
4,INDP,Industry recode for 2013 and later based on 20...,31.322720
5,WKL,When last worked,29.493317
6,MSP,"Married, spouse present/spouse absent",27.743804
7,MARHYP,Year last married,26.990797
8,SCHL,Educational attainment,25.316544
9,WKHP,Usual hours worked per week past 12 months,24.992735


In [711]:
person_data.columns

Index(['RT', 'SERIALNO', 'SPORDER', 'PUMA', 'ST', 'ADJINC', 'PWGTP', 'AGEP',
       'CIT', 'CITWP',
       ...
       'pwgtp71', 'pwgtp72', 'pwgtp73', 'pwgtp74', 'pwgtp75', 'pwgtp76',
       'pwgtp77', 'pwgtp78', 'pwgtp79', 'pwgtp80'],
      dtype='object', length=284)

In [712]:
person_data['SERIALNO']

0           315
1           315
2           315
3           315
4           315
5          1408
6          1408
7          1408
8          1508
9          1508
10         1508
11         1508
12         1516
13         1905
14         1905
15         1905
16         2205
17         2582
18         2582
19         2582
20         2582
21         2582
22         2582
23         2582
24         2582
25         2975
26         2975
27         2975
28         3513
29         3548
         ...   
6757    1499164
6758    1499164
6759    1499164
6760    1499164
6761    1499164
6762    1499164
6763    1499164
6764    1499164
6765    1499164
6766    1499814
6767    1499814
6768    1499988
6769    1499988
6770    1499988
6771    1499988
6772    1499988
6773    1499988
6774    1499988
6775    1499988
6776    1499988
6777    1500349
6778    1500349
6779    1500608
6780    1500776
6781    1500776
6782    1500776
6783    1500776
6784    1500776
6785    1500776
6786    1500776
Name: SERIALNO, dtype: i

In [734]:
merged_data = pd.merge(housing_data, person_data, how='outer', on=['SERIALNO', 'PUMA', 'ST', 'ADJINC'], suffixes=['_H', '_P'])
merged_data.WGTP = merged_data.WGTP / ((merged_data.NP == 0) + merged_data.NP)

In [735]:
merged_data

,RT_H,SERIALNO,DIVISION,PUMA,REGION,ST,ADJHSG,ADJINC,WGTP,NP,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,H,77,9,400,4,2,1000000,1008425,22.000000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H,315,9,200,4,2,1000000,1008425,32.200000,5,...,165.0,156.0,44.0,43.0,243.0,213.0,56.0,138.0,243.0,163.0
2,H,315,9,200,4,2,1000000,1008425,32.200000,5,...,178.0,141.0,47.0,45.0,253.0,227.0,39.0,134.0,278.0,144.0
3,H,315,9,200,4,2,1000000,1008425,32.200000,5,...,289.0,251.0,68.0,71.0,438.0,386.0,64.0,230.0,396.0,230.0
4,H,315,9,200,4,2,1000000,1008425,32.200000,5,...,222.0,194.0,56.0,63.0,350.0,344.0,60.0,217.0,322.0,180.0
5,H,315,9,200,4,2,1000000,1008425,32.200000,5,...,192.0,143.0,56.0,52.0,304.0,248.0,52.0,165.0,255.0,176.0
6,H,807,9,300,4,2,1000000,1008425,1303.000000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,H,1408,9,200,4,2,1000000,1008425,16.666667,3,...,62.0,79.0,92.0,55.0,46.0,51.0,50.0,91.0,56.0,40.0
8,H,1408,9,200,4,2,1000000,1008425,16.666667,3,...,73.0,90.0,108.0,69.0,65.0,59.0,57.0,114.0,67.0,49.0
9,H,1408,9,200,4,2,1000000,1008425,16.666667,3,...,105.0,184.0,205.0,116.0,137.0,110.0,117.0,320.0,115.0,181.0
